In [1]:
#%pip install tensorflow

In [2]:
import pandas as pd
from data import get_rating_matrix, get_interactions, get_job_data, get_job_interactions_matrix, map_ids
from pyspark_ALS import pyspark_ALS
import numpy as np

In [3]:
lookback = 90
rating_matrix = pd.read_csv('./data/ratings-filtered-%s-days.csv' % lookback, index_col=0)
item_feature_df =pd.read_csv('./data/item_feature-filtered-%s-days.csv' % lookback, index_col=0)
long_tail_items = item_feature_df[item_feature_df.feature == 'long_tail'].iidx.to_numpy()
rating_matrix
# %%
rating_matrix_for_pyspark = rating_matrix.drop(columns=['industry', 'county', 'userId', 'itemId', 'timestamp', 'applicationdeadline', 'published', 'state'])
rating_matrix_for_pyspark

,rating,uidx,iidx
126,1,0,0
3753501,1,1,1
3753468,1,2,1
3753430,1,3,1
3753411,1,4,1
...,...,...,...
3530538,5,2276,760
3526731,5,2848,754
3525838,5,1993,755
3525790,5,2877,755


In [4]:
pandas_user_recs, pandas_test, train = pyspark_ALS(rating_matrix_for_pyspark.rename(columns={'uidx': 'userId', 'iidx': 'itemId'}), 50, long_tail_items, implicit=True)

Start training
Num models to be tested:  16
Trining Done
Creating recommendations
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 3593
**Best Model**
  Rank: 150
  MaxIter: 5
  RegParam: 0.01
  Summary Stats
  RMSE test 1.6549309791261737
  Precision = 0.9389368215975509
  Recall = 0.003238830015859092
  NDCG = 0.9463282535257757
  Summary Popularity Bias
  ARP@10 24.598747564709182
  APLT@10 0.040801558586139375
  ACLT@10 0.40801558586139713


C:\Users\perni\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)
C:\Users\perni\Documents\NTNU\Master\Master-Popularity-Bias-In-JobRecs\pyspark_ALS.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_list[global_user] = testItems
C:\Users\perni\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [5]:
pandas_user_recs.to_pickle('./data/outputs/pyspark-als.preds.pkl')
train.to_pickle('./data/outputs/pyspark-als-train.pkl')
pandas_test.to_pickle('./data/outputs/pyspark-als-test.pkl')

# %%
#TODO: REMOVE


#pandas_user_recs = pd.read_pickle('./data/outputs/pyspark-als.preds.pkl')
#train = pd.read_pickle('./data/outputs/pyspark-als-train.pkl')
#pandas_test = pd.read_pickle('./data/outputs/pyspark-als-test.pkl')

In [6]:
#from xQuAD.xQuAD import RankerXQuad
users = list(np.unique(rating_matrix['uidx'].values))
items = list(np.unique(rating_matrix['iidx'].values))

# from sklearn.model_selection import KFold

# kfold = KFold(3, True)

# for train, test in kfold.split(train):

In [8]:
%time
from xQuAD.xQuAD import RankerXQuad
from xQuAD.Model import save_obj
import os

lmbdas = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#lmbdas = [0.4]

predictions = pandas_user_recs.pivot(index='userId', columns='itemId', values = 'rating').fillna(0)
predictions = predictions.reindex(index=users, columns=items, fill_value=0).values
models = {}
model = RankerXQuad(users = users, items = items, observed_relevance = train, unobserved_relevance = pandas_test, item_field = 'itemId', rating_field='rating', user_field = 'userId', item_feature_df=item_feature_df)


for lmbda in lmbdas:
    print("Training with ", {'lambda': lmbda})
    model.set_predictions(predictions)
    model.rerank(type='smooth', lmbda=lmbda, k=10, rmax=100)
    scores = model.get_predictions()
    save_obj(scores, os.path.join('./data/outputs/scores/pyspark/reranker-pyspark' + str(lmbda) + '_scores.pkl'))
    model.test()

    metrics = model.get_metrics()
    save_obj(metrics, os.path.join('./data/outputs/metrics/pyspark/reranker-pyspark' + str(lmbda) + '_metrics.pkl'))
    model.show_metrics()
    models[lmbda] = model

Wall time: 0 ns
Initializing user and  item lists
Initializing observed, unobserved, and predicted relevance scores
Initializing item popularity lists
Initializing metrics
Training with  {'lambda': 0.0}
0.0
Computing metrics for user 4365 / 4365365654365 164 / 43654365/ 43654365/ 4365/ 436543654365 436543651039 / 436543651610 / 43651679 / 4365 4365/ 4365436543654365436543652211 / 43654365 436543652356 / 43654365/ 43654365 2559 / 4365436543652725 / 436543654365 43652959 / 4365 43654365 436543653322 / 43653444 / 436543654365/ 43654365 4365/ 4365
4365
2876.0
Precision: 0.0285 
Recall: 0.132 
NDCG: 0.0905 
Hit Rate: 0.2447 
Avg Popularity: 21.6113 
Category Diversity: 0.0 
Novelty: 7.9618 
Item Coverage: 0.57 
User Coverage: 0.2447 
ARP 21.6113 
APLT 0.0659 
ACLT 0.6589
Training with  {'lambda': 0.1}
0.1
Computing metrics for user 4365 / 436536565 4365 4365 4365436543654365/ 4365436543651021 / 4365436543651405 / 43654365 1441 / 43654365 / 4365/ 436543651726 / 43654365 43652345 / 4365436543

C:\Users\perni\Documents\NTNU\Master\Master-Popularity-Bias-In-JobRecs\xQuAD\xQuAD.py:40: RuntimeWarning: invalid value encountered in subtract
  user_scores = (user_scores - min(user_scores)) / (max(user_scores) - min(user_scores))
C:\Users\perni\Documents\NTNU\Master\Master-Popularity-Bias-In-JobRecs\xQuAD\xQuAD.py:40: RuntimeWarning: invalid value encountered in true_divide
  user_scores = (user_scores - min(user_scores)) / (max(user_scores) - min(user_scores))


In [ ]:
models[0.4].test()

In [ ]:
model.show_metrics()
# %%
# %%
